### Installation

In [ ]:
#! pip install matplotlib
#!pip install folium
#!pip install altair
#!pip install branca

### Imports

In [1]:
import pandas as pd
from math import sqrt
import folium
from folium import plugins
from folium.features import FeatureGroup
import altair as alt

### Process database 1

In [2]:
# Read 'xlsx' file as dataframe
mal = pd.read_excel('../data/processed/maltratos_hospitales.xlsx', index_col=None)

# Select columns by index
cols = mal.iloc[:, 1:11]
# Sum up selected columns and create a new column
mal['Total'] = cols.sum(axis=1)

# Create a copy of dataframe without certain columns
mal_copy = mal.drop(columns=['Long', 'Lat', 'Hospital', 'Total'])

# Create a new column with index values
mal_copy.insert(loc=0, column='Id', value=mal.index)

# Modify values from selected column
mal_copy['Id'] = 'Hospital' + mal_copy['Id'].astype(str)

# Set specified column as index
mal_copy = mal_copy.set_index('Id')

# Transpose dataframe
mal_tr = mal_copy.transpose()

# Create a list
tipos = ['Maltrato1', 'Maltrato2', 'Maltrato3', 'Maltrato4', 'Maltrato5', 
         'Maltrato6', 'Maltrato7', 'Maltrato8', 'Maltrato9', 'Maltrato10']
# Add list as a column
mal_tr.insert(loc=0, column='Tipos', value=tipos)

# Create a list
maltratos = ['Te llamaron por sobrenombres o diminutivos (mamita, niña, gorda, negra, etc)', 
             'Se burlaron de ti', 
             'Te hablaron en voz alta de modo grosero', 
             'Ignoraron algo que les pediste',
             'Te dijeron que no explícitamente a algo que pediste', 
             'Te dieron golpes, sacudidas, empujones', 
             'Te dijeron que estabas poniendo en riesgo a tu hijo/a', 
             'Te culparon por situaciones ocurridas en el nacimiento', 
             'Te amenazaron', 
             'Otro tipo de maltrato']
# Add list as a column
mal_tr.insert(loc=1, column='Maltratos', value=maltratos)

# Reset index
mal_tr = mal_tr.reset_index(drop=True).rename_axis('', axis='columns')

# Create a list with the index
mal_count = list(mal.index)

# List with columns names
columns_list = list(mal_tr)

# List with column names
mal_hospitals = mal['Hospital']

### Process database 2

In [3]:
# Read 'xlsx' file as dataframe
sec = pd.read_excel('../data/processed/secuelas_hospitales.xlsx', index_col=None)

# Select columns by index
cols = sec.iloc[:, 1:12]
# Sum up selected columns and create a new column
sec['Total'] = cols.sum(axis=1)

# Create a copy of dataframe without certain columns
sec_copy = sec.drop(columns=['Long', 'Lat', 'Hospital', 'Total'])

# Create a new column with index values
sec_copy.insert(loc=0, column='Id', value=sec.index)

# Modify values from selected column
sec_copy['Id'] = 'Hospital' + sec_copy['Id'].astype(str)

# Set specified column as index
sec_copy = sec_copy.set_index('Id')

# Transpose dataframe
sec_tr = sec_copy.transpose()

# Create a list
tipos = ['Consecuencia1', 'Consecuencia2', 'Consecuencia3', 'Consecuencia4', 'Consecuencia5', 
         'Consecuencia6', 'Consecuencia7', 'Consecuencia8', 'Consecuencia9', 'Consecuencia10',
         'Consecuencia11']
# Add list as a column
sec_tr.insert(loc=0, column='Tipos', value=tipos)

# Create a list
consecuencias = ['Dificultad para conectar emocionalmente con tu hijo/a', 
             'Problemas en la lactancia', 
             'Rechazo al contacto con tu hijo/a', 
             'Miedo a no ser "buena madre',
             'Miedo frente al estado de salud de tu hijo/a', 
             'Rechazo frente a las tareas de cuidado y crianza de tu hijo/a', 
             'Desgano y tristeza en situaciones compartidas con tu hijo/a (lactancia, juegos, etc)', 
             'Tener dudas o temores sobre volver a salir embarazada/parir', 
             'Negarte rotundamente a volver a salir embarazada/parir', 
             'Desconfianza en el sistema médico y los/as profesionales de la medicina',
             'Miedo por tu bienestar y/o de tu hijo/a']

# Add list as a column
sec_tr.insert(loc=1, column='Consecuencias', value=consecuencias)

# Reset index
sec_tr = sec_tr.reset_index(drop=True).rename_axis('', axis='columns')

# Create a list with the index
sec_count = list(sec.index)

# List with columns names
columns_list = list(sec_tr)

# List with column names
sec_hospitals = sec['Hospital']

### Custom map

In [5]:
# Create a map at specified location
m = folium.Map(location=[22.037635528256533, -79.36879425670136], zoom_start=7, 
               tiles=None)

# Add layer and set dark as default mode
folium.TileLayer(tiles="CartoDB dark_matter", name="Modo oscuro", attr= '''
               &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> 
               contributors &copy; <a href="https://carto.com/attributions">CARTO</a>
               ''').add_to(m)

# Add layer for light mode
folium.TileLayer(tiles='https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png', 
                 name="Modo claro", attr= '''
                 &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> 
                 contributors &copy; <a href="https://carto.com/attributions">CARTO</a>
                ''', overlay=False).add_to(m)

# Add fullscreen option
plugins.Fullscreen(
    position="topright",
    title="Pantalla completa",
    title_cancel="Salir de pantalla completa",
    force_separate_button=True,
).add_to(m)

# Add a group
maltratos = FeatureGroup(name="Maltratos")
maltratos.add_to(m)

# Add a group
secuelas = FeatureGroup(name="Secuelas", show=False)
secuelas.add_to(m)

# Add the option to switch layers
folium.LayerControl('topright', collapsed= False).add_to(m)

# Empty list for bar charts
charts_mal = []
# Create a bar chart for each hospital column
for i in range(len(mal_count)):
    charts_mal.append("chart"+ f'{i}')
    charts_mal[i] = alt.Chart(mal_tr).mark_bar(color='darkviolet', size=20).encode(
            alt.X(f'Hospital{i}', title='Encuestadas', 
                  scale=alt.Scale(domain=[0, 30])),
            alt.Y(columns_list[0], title='Tipos de maltrato', 
                  sort=alt.EncodingSortField(field=columns_list[i], op="count", order='ascending')),
            tooltip=[
                  alt.Tooltip('Maltratos', title="Tipo de maltrato"),
                  alt.Tooltip(f'Hospital{i}', title="Casos")]
                  ).interactive().properties(title=f'{mal_hospitals[i]}', 
                                                              width=350, height=300)
    # Save chart as 'json'
    charts_mal[i].to_json()

# Add circle markers for each row in df
k = 0
for j, r in mal.iterrows():
    folium.CircleMarker([r['Long'], r['Lat']],
    radius=sqrt(r['Total']),
    color='#9b00ff',
    fill_color='#9b00ff',
    fill_opacity=(0.5),
    tooltip=r['Hospital'],
    # Add popups with bar charts
    popup=folium.Popup(max_width=450, max_heigth=400).add_child(folium.VegaLite(charts_mal[k], width=450, height=300))
    ).add_to(maltratos)
    k += 1

# Empty list for bar charts
charts_sec = []
# Create a bar chart for each hospital column
for i in range(len(sec_count)):
    charts_sec.append("chart"+ f'{i}')
    charts_sec[i] = alt.Chart(sec_tr).mark_bar(color='indigo', size=20).encode(
            alt.X(f'Hospital{i}', title='Encuestadas', 
                  scale=alt.Scale(domain=[0, 50])),
            alt.Y(columns_list[0], title='Tipos de secuela', 
                  sort=alt.EncodingSortField(field=columns_list[i], op="count", order='ascending')),
            tooltip=[
                  alt.Tooltip('Consecuencias', title="Tipo de secuela"),
                  alt.Tooltip(f'Hospital{i}', title="Casos")]
                  ).interactive().properties(title=f'{sec_hospitals[i]}', 
                                                              width=350, height=300)
    # Save chart as 'json'
    charts_sec[i].to_json()

# Add circle markers for each row in df
k = 0
for j, r in sec.iterrows():
    folium.CircleMarker([r['Long'], r['Lat']],
    radius=sqrt(r['Total']),
    color='#5F9EA0',
    fill_color='#40E0D0',
    fill_opacity=(0.5),
    tooltip=r['Hospital'],
    # Add popups with bar charts
    popup=folium.Popup(max_width=450, max_heigth=400).add_child(folium.VegaLite(charts_sec[k], width=450, height=300))
    ).add_to(secuelas)
    k += 1

# Save map as 'html'
m.save("../index.html")

# Show map
m